In [1]:
import pandas as pd

# Load the datasets (Assuming CSV files, you can modify paths accordingly)
trips_df = pd.read_csv(r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\trips_db\fact_trips.csv")
passenger_summary_df = pd.read_csv(r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\trips_db\fact_passenger_summary.csv")
dim_date_df = pd.read_csv(r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\trips_db\dim_date.csv")
city_target_passenger_rating = pd.read_csv(r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\targets_db\city_target_passenger_rating.csv")
target_trips_df = pd.read_csv(r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\targets_db\monthly_target_trips.csv")
monthly_target_new_passenger = pd.read_csv(r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\targets_db\monthly_target_new_passengers.csv")
dim_city_df = pd.read_csv(r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\trips_db\dim_city.csv")
dim_repeat_trip_distribution_df = pd.read_csv(r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\trips_db\dim_repeat_trip_distribution.csv")

# Merging data to include the month name in the analysis
df = pd.merge(trips_df, dim_date_df[['date', 'month_name']], how='left', on='date')

# Metric 1: Total Trips
total_trips = df.groupby(['city_id', 'month_name']).agg(
    total_trips=('trip_id', 'count')
).reset_index()

# Metric 2: Total Revenue (Fare)
total_revenue = df.groupby(['city_id', 'month_name']).agg(
    monthly_revenue=('fare_amount', 'sum')
).reset_index()

# Metric 3: Total Distance Travelled (Note: Use exact column name for distance travelled)
total_distance = df.groupby(['city_id', 'month_name']).agg(
    total_distance=('distance_travelled(km)', 'sum')  # Correct column name with the parentheses
).reset_index()

# Metric 4: Average Passenger Rating
avg_passenger_rating = df.groupby(['city_id', 'month_name']).agg(
    avg_passenger_rating=('passenger_rating', 'mean')
).reset_index()

# Metric 5: Average Driver Rating
avg_driver_rating = df.groupby(['city_id', 'month_name']).agg(
    avg_driver_rating=('driver_rating', 'mean')
).reset_index()

# Metric 6: New Trips (Count of trips by new passengers)
new_trips = df[df['passenger_type'] == 'new'].groupby(['city_id', 'month_name']).agg(
    new_trips=('trip_id', 'count')
).reset_index()

# Metric 7: Repeated Trips (Count of trips by repeated passengers)
repeated_trips = df[df['passenger_type'] == 'repeated'].groupby(['city_id', 'month_name']).agg(
    repeated_trips=('trip_id', 'count')
).reset_index()

# Metric 8: Average Trip Distance
avg_trip_distance = df.groupby(['city_id', 'month_name']).agg(
    avg_trip_distance=('distance_travelled(km)', 'mean')  # Correct column name with parentheses
).reset_index()

# Metric 9: Maximum and Minimum Trip Distance
max_min_distance = df.groupby(['city_id', 'month_name']).agg(
    max_trip_distance=('distance_travelled(km)', 'max'),
    min_trip_distance=('distance_travelled(km)', 'min')
).reset_index()

# Metric 10: Total Passengers, New Passengers, Repeat Passengers
passenger_summary_df['month_name'] = pd.to_datetime(passenger_summary_df['month'], format='%Y-%m-%d').dt.strftime('%B')
passenger_summary = passenger_summary_df.groupby(['city_id', 'month_name']).agg(
    total_passengers=('total_passengers', 'sum'),
    new_passengers=('new_passengers', 'sum'),
    repeat_passengers=('repeat_passengers', 'sum')
).reset_index()

# Merge all the metrics
final_df = total_trips.merge(total_revenue, on=['city_id', 'month_name'])
final_df = final_df.merge(total_distance, on=['city_id', 'month_name'])
final_df = final_df.merge(avg_passenger_rating, on=['city_id', 'month_name'])
final_df = final_df.merge(avg_driver_rating, on=['city_id', 'month_name'])
final_df = final_df.merge(new_trips, on=['city_id', 'month_name'])
final_df = final_df.merge(repeated_trips, on=['city_id', 'month_name'])
final_df = final_df.merge(avg_trip_distance, on=['city_id', 'month_name'])
final_df = final_df.merge(max_min_distance, on=['city_id', 'month_name'])
final_df = final_df.merge(passenger_summary, on=['city_id', 'month_name'])

# Add the monthly trip target data
# Ensure correct date format for `target_trips_df['month']`
# Try to handle both formats: 'YYYY-MM-DD' or 'DD-MM-YYYY'
target_trips_df['month_name'] = pd.to_datetime(target_trips_df['month'], errors='coerce', dayfirst=True).dt.strftime('%B')
final_df = final_df.merge(target_trips_df[['city_id', 'month_name', 'total_target_trips']], on=['city_id', 'month_name'], how='left')

# Metric 11: Trip Target Achievement (percentage)
final_df['trip_target_achievement'] = final_df['total_trips'] * 100.0 / final_df['total_target_trips']

# Metric 12: Repeat Passenger Rate
final_df['repeat_passenger_rate'] = final_df['repeat_passengers'] * 100.0 / final_df['total_passengers']

# File 1: Grouped by city_id
final_df_city = final_df.groupby('city_id').agg(
    total_trips=('total_trips', 'sum'),
    monthly_revenue=('monthly_revenue', 'sum'),
    total_distance=('total_distance', 'sum'),
    avg_passenger_rating=('avg_passenger_rating', 'mean'),
    avg_driver_rating=('avg_driver_rating', 'mean'),
    new_trips=('new_trips', 'sum'),
    repeated_trips=('repeated_trips', 'sum'),
    avg_trip_distance=('avg_trip_distance', 'mean'),
    max_trip_distance=('max_trip_distance', 'max'),
    min_trip_distance=('min_trip_distance', 'min'),
    total_passengers=('total_passengers', 'sum'),
    new_passengers=('new_passengers', 'sum'),
    repeat_passengers=('repeat_passengers', 'sum'),
    repeat_passenger_rate=('repeat_passenger_rate', 'mean'),
    trip_target_achievement=('trip_target_achievement', 'mean')
).reset_index()

# File 2: Grouped by month_name
final_df_month = final_df.groupby('month_name').agg(
    total_trips=('total_trips', 'sum'),
    monthly_revenue=('monthly_revenue', 'sum'),
    total_distance=('total_distance', 'sum'),
    avg_passenger_rating=('avg_passenger_rating', 'mean'),
    avg_driver_rating=('avg_driver_rating', 'mean'),
    new_trips=('new_trips', 'sum'),
    repeated_trips=('repeated_trips', 'sum'),
    avg_trip_distance=('avg_trip_distance', 'mean'),
    max_trip_distance=('max_trip_distance', 'max'),
    min_trip_distance=('min_trip_distance', 'min'),
    total_passengers=('total_passengers', 'sum'),
    new_passengers=('new_passengers', 'sum'),
    repeat_passengers=('repeat_passengers', 'sum'),
    repeat_passenger_rate=('repeat_passenger_rate', 'mean'),
    trip_target_achievement=('trip_target_achievement', 'mean')
).reset_index()

# Save the results to CSV
final_df_city.to_csv('goodcabs_analysis_by_city.csv', index=False)
final_df_month.to_csv('goodcabs_analysis_by_month.csv', index=False)

# Optionally, print the first few rows of the files
print(final_df_city.head())
print(final_df_month.head())


  city_id  total_trips  monthly_revenue  total_distance  avg_passenger_rating  \
0    AP01        28366          8018282          639765              8.434342   
1    CH01        38981         11058401          916783              7.973127   
2    GJ01        54843          6431599          603122              6.420498   
3    GJ02        32026          3797200          368867              6.616081   
4    KA01        16238          4054745          267877              8.704889   

   avg_driver_rating  new_trips  repeated_trips  avg_trip_distance  \
0           8.987120      12747           15619          22.554778   
1           7.722465      18908           20073          23.523374   
2           6.589763      11626           43217          10.995752   
3           6.648147      10127           21899          11.519337   
4           8.978083      11681            4557          16.493180   

   max_trip_distance  min_trip_distance  total_passengers  new_passengers  \
0              

In [2]:
import pandas as pd

# Load the datasets with date parsing
trips_df = pd.read_csv(
    r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\trips_db\fact_trips.csv",
    parse_dates=["date"], dayfirst=True
)
dim_date_df = pd.read_csv(
    r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\trips_db\dim_date.csv",
    parse_dates=["date", "start_of_month"], dayfirst=True
)
dim_repeat_trip_distribution_df = pd.read_csv(
    r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\trips_db\dim_repeat_trip_distribution.csv",
    parse_dates=["month"], dayfirst=True
)

# Clean and preprocess the datasets
# Remove "-Trips" suffix and convert `trip_count` to numeric
dim_repeat_trip_distribution_df['trip_count'] = dim_repeat_trip_distribution_df['trip_count'].str.replace('-Trips', '').astype(int)

# Ensure all numeric columns in fact_trips are properly formatted
trips_df['distance_travelled(km)'] = pd.to_numeric(trips_df['distance_travelled(km)'], errors='coerce')
trips_df['fare_amount'] = pd.to_numeric(trips_df['fare_amount'], errors='coerce')

# Explicitly ensure date columns are in datetime format
trips_df['date'] = pd.to_datetime(trips_df['date'], dayfirst=True)
dim_date_df['date'] = pd.to_datetime(dim_date_df['date'], dayfirst=True)

# Primary Analysis Functions

# 1. Top and Bottom Performing Cities by Total Trips
def top_bottom_cities(trips_df):
    city_summary = trips_df.groupby('city_id').agg(
        total_trips=('trip_id', 'count')
    ).reset_index()
    city_summary = city_summary.sort_values(by='total_trips', ascending=False)
    top_cities = city_summary.head(3)
    bottom_cities = city_summary.tail(3)
    return top_cities, bottom_cities

# 2. Average Fare per Trip by City
def avg_fare_per_trip(trips_df):
    avg_fare = trips_df.groupby('city_id').agg(
        total_fare=('fare_amount', 'sum'),
        total_trips=('trip_id', 'count')
    ).reset_index()
    avg_fare['avg_fare_per_trip'] = avg_fare['total_fare'] / avg_fare['total_trips']
    return avg_fare[['city_id', 'avg_fare_per_trip']]

# 3. Peak and Low Demand Months by City
def peak_low_demand(trips_df, dim_date_df):
    trips_df = pd.merge(trips_df, dim_date_df[['date', 'month_name']], on='date', how='left')
    monthly_summary = trips_df.groupby(['city_id', 'month_name']).agg(
        total_trips=('trip_id', 'count')
    ).reset_index()
    peak_months = monthly_summary.loc[monthly_summary.groupby('city_id')['total_trips'].idxmax()]
    low_months = monthly_summary.loc[monthly_summary.groupby('city_id')['total_trips'].idxmin()]
    return peak_months, low_months

# 4. Weekend vs Weekday Trip Demand
def weekend_vs_weekday_analysis(trips_df, dim_date_df):
    trips_df = pd.merge(trips_df, dim_date_df[['date', 'day_type']], on='date', how='left')
    weekday_weekend = trips_df.groupby(['city_id', 'day_type']).agg(
        total_trips=('trip_id', 'count')
    ).reset_index()
    return weekday_weekend

# 5. Repeat Passenger Frequency and Contribution
def repeat_passenger_analysis(dim_repeat_trip_distribution_df):
    repeat_freq = dim_repeat_trip_distribution_df.groupby(['city_id']).agg(
        total_repeat_passengers=('repeat_passenger_count', 'sum'),
        total_trips=('trip_count', 'sum')
    ).reset_index()
    repeat_freq['repeat_passenger_rate'] = (
        repeat_freq['total_repeat_passengers'] * 100.0 / repeat_freq['total_trips']
    )
    return repeat_freq

# Run the analysis
analysis_results = {
    "top_bottom_cities": top_bottom_cities(trips_df),
    "avg_fare_per_trip": avg_fare_per_trip(trips_df),
    "peak_low_demand": peak_low_demand(trips_df, dim_date_df),
    "weekend_vs_weekday": weekend_vs_weekday_analysis(trips_df, dim_date_df),
    "repeat_passenger_analysis": repeat_passenger_analysis(dim_repeat_trip_distribution_df)
}

# Save results to CSV
for key, result in analysis_results.items():
    if isinstance(result, tuple):  # Handle metrics with high/low segmentation
        result[0].to_csv(f"{key}_high.csv", index=False)
        result[1].to_csv(f"{key}_low.csv", index=False)
    else:
        result.to_csv(f"{key}.csv", index=False)

print("Analysis completed and results saved.")




Analysis completed and results saved.


In [3]:
import pandas as pd

# Load the datasets
trips_df = pd.read_csv(r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\trips_db\fact_trips.csv", parse_dates=["date"],dayfirst=True)
dim_date_df = pd.read_csv(r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\trips_db\dim_date.csv", parse_dates=["date", "start_of_month"])
dim_repeat_trip_distribution_df = pd.read_csv(r"C:\Users\anshi\OneDrive\Desktop\Data analyst\Resume project challange_13\resources\datasets\csv_files\trips_db\dim_repeat_trip_distribution.csv", parse_dates=["month"])

# Convert 'date' columns to datetime (in case they're not already in the correct format)
trips_df['date'] = pd.to_datetime(trips_df['date'], errors='coerce')
dim_date_df['date'] = pd.to_datetime(dim_date_df['date'], errors='coerce')

# Clean and preprocess the datasets
# Remove "-Trips" suffix and convert `trip_count` to numeric
dim_repeat_trip_distribution_df['trip_count'] = dim_repeat_trip_distribution_df['trip_count'].str.replace('-Trips', '').astype(int)

# Ensure all numeric columns in fact_trips are properly formatted
trips_df['distance_travelled(km)'] = pd.to_numeric(trips_df['distance_travelled(km)'], errors='coerce')
trips_df['fare_amount'] = pd.to_numeric(trips_df['fare_amount'], errors='coerce')

# Primary Analysis Functions

# 1. Top and Bottom Performing Cities by Total Trips
def top_bottom_cities(trips_df):
    city_summary = trips_df.groupby('city_id').agg(
        total_trips=('trip_id', 'count')
    ).reset_index()
    city_summary = city_summary.sort_values(by='total_trips', ascending=False)
    top_cities = city_summary.head(3)
    bottom_cities = city_summary.tail(3)
    return top_cities, bottom_cities

# 2. Average Fare per Trip by City
def avg_fare_per_trip(trips_df):
    avg_fare = trips_df.groupby('city_id').agg(
        total_fare=('fare_amount', 'sum'),
        total_trips=('trip_id', 'count')
    ).reset_index()
    avg_fare['avg_fare_per_trip'] = avg_fare['total_fare'] / avg_fare['total_trips']
    return avg_fare[['city_id', 'avg_fare_per_trip']]

# 3. Peak and Low Demand Months by City
def peak_low_demand(trips_df, dim_date_df):
    trips_df = pd.merge(trips_df, dim_date_df[['date', 'month_name']], on='date', how='left')
    monthly_summary = trips_df.groupby(['city_id', 'month_name']).agg(
        total_trips=('trip_id', 'count')
    ).reset_index()
    peak_months = monthly_summary.loc[monthly_summary.groupby('city_id')['total_trips'].idxmax()]
    low_months = monthly_summary.loc[monthly_summary.groupby('city_id')['total_trips'].idxmin()]
    return peak_months, low_months

# 4. Weekend vs Weekday Trip Demand
def weekend_vs_weekday_analysis(trips_df, dim_date_df):
    trips_df = pd.merge(trips_df, dim_date_df[['date', 'day_type']], on='date', how='left')
    weekday_weekend = trips_df.groupby(['city_id', 'day_type']).agg(
        total_trips=('trip_id', 'count')
    ).reset_index()
    return weekday_weekend

# 5. Repeat Passenger Frequency and Contribution
def repeat_passenger_analysis(dim_repeat_trip_distribution_df):
    repeat_freq = dim_repeat_trip_distribution_df.groupby(['city_id']).agg(
        total_repeat_passengers=('repeat_passenger_count', 'sum'),
        total_trips=('trip_count', 'sum')
    ).reset_index()
    repeat_freq['repeat_passenger_rate'] = (
        repeat_freq['total_repeat_passengers'] * 100.0 / repeat_freq['total_trips']
    )
    return repeat_freq

# Run the analysis
analysis_results = {
    "top_bottom_cities": top_bottom_cities(trips_df),
    "avg_fare_per_trip": avg_fare_per_trip(trips_df),
    "peak_low_demand": peak_low_demand(trips_df, dim_date_df),
    "weekend_vs_weekday": weekend_vs_weekday_analysis(trips_df, dim_date_df),
    "repeat_passenger_analysis": repeat_passenger_analysis(dim_repeat_trip_distribution_df)
}

# Print the results to the console
for key, result in analysis_results.items():
    if isinstance(result, tuple):  # Handle metrics with high/low segmentation
        print(f"\n{key} - High:\n", result[0])
        print(f"\n{key} - Low:\n", result[1])
    else:
        print(f"\n{key}:\n", result)



top_bottom_cities - High:
   city_id  total_trips
7    RJ01        76888
9    UP01        64299
2    GJ01        54843

top_bottom_cities - Low:
   city_id  total_trips
0    AP01        28366
8    TN01        21104
4    KA01        16238

avg_fare_per_trip:
   city_id  avg_fare_per_trip
0    AP01         282.672284
1    CH01         283.686950
2    GJ01         117.272925
3    GJ02         118.566165
4    KA01         249.707168
5    KL01         335.245079
6    MP01         179.838609
7    RJ01         483.918128
8    TN01         166.982183
9    UP01         147.180376

peak_low_demand - High:
    city_id month_name  total_trips
0     AP01      April         1033
6     CH01      April         1470
12    GJ01      April         1903
21    GJ02       June         1152
27    KA01       June          612
30    KL01      April         1947
36    MP01      April         1603
46    RJ01      March         3007
51    TN01       June          762
57    UP01       June         2247

peak_low_

In [ ]:
def factors_influencing_repeat_passenger_rate(trips_df, repeat_passenger_data):
    # Merge the repeat passenger data with the trips dataset
    combined_data = pd.merge(trips_df, repeat_passenger_data, on='city_id', how='left')

    # Calculate correlation between factors and repeat passenger rate
    correlation_data = combined_data[['avg_fare_per_trip', 'total_trips', 'avg_passenger_rating', 'repeat_passenger_rate']].corr()

    return correlation_data

# Call the function
repeat_factors_correlation = factors_influencing_repeat_passenger_rate(trips_df, repeat_passenger_analysis)
print(repeat_factors_correlation)


In [ ]:
def tourism_business_demand_impact(trips_df, tourism_cities):
    # Filter trips data for tourism-focused cities
    tourism_data = trips_df[trips_df['city_id'].isin(tourism_cities)]
    
    # Aggregate trips by month and city to observe seasonal trends
    tourism_monthly_data = tourism_data.groupby(['city_id', 'month_name']).agg(
        total_trips=('trip_id', 'count')
    ).reset_index()

    # You can also add analysis of peak months for tourism cities
    tourism_peak_months = tourism_monthly_data.loc[tourism_monthly_data.groupby('city_id')['total_trips'].idxmax()]

    return tourism_monthly_data, tourism_peak_months

# Example of tourism cities (should be provided based on actual categorization)
tourism_cities = ['AP01', 'CH01', 'TN01']
tourism_data, tourism_peak_months = tourism_business_demand_impact(trips_df, tourism_cities)

# Print the results
print(tourism_data)
print(tourism_peak_months)


In [ ]:
def monthly_target_achievement(trips_df, target_trips_df):
    # Merge the target data with actual trips data
    merged_data = pd.merge(trips_df, target_trips_df[['city_id', 'month_name', 'total_target_trips']], on=['city_id', 'month_name'], how='left')
    
    # Calculate achievement percentage for each city and month
    merged_data['trip_target_achievement'] = (merged_data['total_trips'] / merged_data['total_target_trips']) * 100
    
    # Identify if the target was met, exceeded, or missed
    merged_data['target_status'] = merged_data['trip_target_achievement'].apply(lambda x: 'Exceeded' if x > 100 else ('Met' if x == 100 else 'Missed'))

    return merged_data[['city_id', 'month_name', 'trip_target_achievement', 'target_status']]

# Run the target achievement analysis
target_achievement_data = monthly_target_achievement(trips_df, target_trips_df)

# Print the results
print(target_achievement_data)


In [ ]:
def repeat_passenger_rate_by_month(trips_df, dim_repeat_trip_distribution_df):
    # Merge the repeat trip distribution data with trips
    repeat_data = pd.merge(trips_df, dim_repeat_trip_distribution_df, on=['city_id', 'month'], how='left')
    
    # Calculate repeat passenger rate (RPR%) for each city and month
    repeat_data['repeat_passenger_rate'] = (repeat_data['repeat_passenger_count'] / repeat_data['trip_count']) * 100
    
    # Identify the highest and lowest repeat passenger rates
    highest_rpr = repeat_data.loc[repeat_data.groupby('city_id')['repeat_passenger_rate'].idxmax()]
    lowest_rpr = repeat_data.loc[repeat_data.groupby('city_id')['repeat_passenger_rate'].idxmin()]

    return highest_rpr[['city_id', 'month', 'repeat_passenger_rate']], lowest_rpr[['city_id', 'month', 'repeat_passenger_rate']]

# Run the analysis
highest_rpr, lowest_rpr = repeat_passenger_rate_by_month(trips_df, dim_repeat_trip_distribution_df)

# Print the results
print("Highest Repeat Passenger Rate (RPR%) by City and Month:")
print(highest_rpr)
print("\nLowest Repeat Passenger Rate (RPR%) by City and Month:")
print(lowest_rpr)


In [ ]:
def analyze_mobility_trends():
    # Example function for analyzing emerging trends, such as electric vehicles
    # We would need external data for electric vehicle adoption rates, and so on
    pass  # Placeholder function for future data integration

# This would need external data collection
analyze_mobility_trends()


In [ ]:
def partnership_opportunities():
    # This would involve gathering foot traffic data and matching it with customer demand patterns
    pass  # Placeholder function for further business analysis

# Placeholder for partnership opportunities function
partnership_opportunities()


In [ ]:
def data_collection_recommendations():
    # Suggest data collection methods for deeper insights
    print("1. Collect customer satisfaction scores and analyze trends.")
    print("2. Track operational efficiency metrics: average wait times, driver ratings.")
    print("3. Monitor market trends: competitor pricing, customer preferences.")
    
# Print the data collection recommendations
data_collection_recommendations()
